In [3]:
import requests
import json
import re

## Can reasoning model be reasonable medical diagnoser

### Step 1 - Use a distilled version of R1

We use Llama8B R1 as a reasoning engine.


#### How to start Llama8B-R1 locally

Use [vllm](https://docs.vllm.ai/en/stable/)

Download a quantized version of the [model](https://huggingface.co/lmstudio-community/DeepSeek-R1-Distill-Llama-8B-GGUF/tree/main)

Run it with the following command 

```shell
vllm serve './DeepSeek-R1-Distill-Llama-8B-Q4_K_M.gguf' --tokenizer deepseek-ai/DeepSeek-R1-Distill-Llama-8B --max-model-len 8000 --enable-auto-tool-choice --tool-call-parser llama3_json
```

You should be able to access it with an API call 

```shell
curl http://localhost:8000/v1/completions     -H "Content-Type: application/json"     -d '{
        "model": "/home/.../.../DeepSeek-R1-Distill-Llama-8B-Q4_K_M.gguf",
        "prompt": "Create a joke with 2 animals of your choice. Think step by step about the animal attributes and what could link them for a joke. ",
        "max_tokens": 7800,
        "temperature": 0.8
    }'

```

In [186]:
model_name = "/home/amor/Downloads/DeepSeek-R1-Distill-Llama-8B-Q4_K_M.gguf"

In [218]:
PROMPT = "Create a joke with 2 animals of your choice. First list famous jokes and try to adapt them."

def call_r1_small(prompt, max_tok=7800):
    config = {
            "model": model_name,
            "prompt": prompt, 
            "max_tokens": max_tok,
            "temperature": 0.8
        }
    
    rez = requests.post("http://localhost:8000/v1/completions",
                        data=json.dumps(config), headers={"Content-Type": "application/json"})
    full_answer = rez.json()["choices"][0]["text"]
    pattern = re.compile("</think>")
    final_answer = full_answer[pattern.search(full_answer).span()[1]:]
    return full_answer, final_answer


In [219]:
full_answer, final_answer = call_r1_small(PROMPT)

In [220]:
final_answer

'\n\n**Joke 1: Classic Adaptation**\n\n"Why did the dog bring a ladder to the cats\' tree?  \nTo help the cat get down!"\n\nThis joke adapts the classic "Why did the chicken cross the road?" joke by swapping the birds with a dog and a cat. The punchline twist adds humor as the dog helps the cat, which is unexpected.\n\n---\n\n**Joke 2: Original**\n\n"Why did the dog ask the cat for directions?  \nBecause the dog wanted to climb the cat tree!"\n\nThis original joke cleverly plays on the classic chicken joke structure, adding a twist with a dog trying to climb a cat\'s tree, highlighting the pets\' differing behaviors.'

In [221]:
joke_prompt = "Find the best LLM prompt to use to create jokes about animals. Ex: Create a joke with 2 animals of your choice, try to adapt already famous jokes."
full_answer, final_answer = call_r1_small(joke_prompt)


In [222]:
print(full_answer)


 Okay, wait, that's a bit vague. Maybe I need to give it more structure. So, the user wants to create a joke where they choose two animals, and then try to adapt a famous joke structure. Hmm, how can I make this better?

Let me start by understanding what the user needs. They want to generate a joke using two animals of their choice, and it should be an adaptation of a famous joke. So, the prompt needs to be clear enough to guide the generation process effectively.

The original example the user gave was: "Create a joke with 2 animals of your choice, try to adapt already famous jokes." Okay, that's a good start, but it might be a bit too vague. Maybe it's missing some structure. To make it better, perhaps I can add some instructions, like specifying the type of famous joke or providing examples of famous joke structures that could be adapted.

Wait, in the example, they specifically mentioned trying to adapt already famous jokes, so maybe the prompt should mention that, for instance, i

In [228]:
NEW_PROMPT = """
To create funny animal jokes, follow this structured approach:

1. **Identify a Classic Structure or Pun**: Start with a well-known joke or pun that you can adapt.

2. **Substitute an Animal**: Replace the original subjects in the joke with an animal that fits naturally into the punchline, ensuring the humor is relevant.

3. **Create a Twist or Pun**: Adapt the punchline to include a pun or a clever twist, ensuring it's understandable and funny.

**Example Jokes**:

- **Original Pun**: "Why did the chicken cross the road?"
- **Substituted Animal**: "Why did the duck cross the road?"
- **Punchline**: "To get to the other side… and avoid the other side’s jokes!"

- **Original Pun**: "What did one fish say to the other?"
- **Substituted Animal**: "What did one fish say to the other?"
- **Punchline**: "I’ll meet you at the nearest sushi joint!"

By applying this method, you can craft a variety of amusing animal jokes that are both clever and easy to understand.

Now create a new joke
"""

In [229]:
a = call_r1_small(NEW_PROMPT, max_tok=7500)

In [230]:
print(a[1])



**Why Did the Duck Cross the Road?**

To avoid the other side's cracks!

**Punchline:** "Why did the duck cross the road? To avoid the other side's cracks!"

This joke cleverly twists the classic structure by replacing the original punchline with a play on "cracks," which humorously references both the act of crossing and the wordplay associated with jokes.


In [232]:
from llama_index.llms.openai_like import OpenAILike

llm = OpenAILike(model=model_name, api_base="http://localhost:8000/v1", api_key="fake", 
                 max_tokens=7500, temperature=0.7)

response = llm.complete("Hello World!")
print(str(response))

 My name is Abhishek, and I'm trying to solve this problem where I need to find the number of possible configurations of a binary tree where each node has two children and the tree has exactly k leaves. Hmm, that's a mouthful. Let me break it down.

First, I need to understand what a binary tree with exactly k leaves looks like. Since each node has two children, the structure of the tree is determined by how many leaves it has. For example, a single node is a leaf itself, so that's one leaf. If it has two children, each of those can be leaves, making it a tree with three nodes and three leaves. Wait, no—if the root has two leaves as children, then that's a tree with three nodes and three leaves, right? Or is that incorrect?

Wait, let me clarify. A leaf node is a node with no children. So, if the root has two children, each of those can be leaves or not. So, the number of leaves depends on how many nodes have no children. If the root has two leaves, then the tree has three nodes and th